# Spelling Normalization with LLM: Experiments on Diderot Encyclopédie

Ollama :https://ollama.com

### Models

https://ollama.com/library/gemma3:4b

## Chargement des données

In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv('data/EDdA_dataset_articles.zip', sep='\t')
df.head() # affiche les 5 première lignes du dataframe

,volume,numero,head,author,artfl_domain,geode_domain,text
0,1,1,Title Page,unsigned,unclassified,Philosophie,"ENCYCLOPÉDIE, DICTIONNAIRE RAISONNÉ DES SCIENC..."
1,1,2,A MONSEIGNEUR LE COMTE D'ARGENSON,Diderot & d'Alembert,unclassified,Philosophie,"A MONSEIGNEUR LE COMTE D'ARGENSON, MINISTRE ET..."
2,1,3,DISCOURS PRÉLIMINAIRE DES EDITEURS,d'Alembert,unclassified,Belles-lettres,DISCOURS PRÉLIMINAIRE DES EDITEURS. L'Encyclop...
3,1,5,"A, a & a",Dumarsais5,Grammaire,Philosophie,"A, a & a s.m. (ordre Encyclopéd. Entend. Scien..."
4,1,6,A,Dumarsais5,unclassified,Philosophie,"A, mot, est 1. la troisieme personne du présen..."


## Sélection aléatoire d'un échantillon de 50 articles

In [17]:
df_sample = df.sample(n=5, random_state=42)

In [18]:
df_sample.head(10) # affiche les 10 première lignes de l'échantillon

,volume,numero,head,author,artfl_domain,geode_domain,text
33382,8,2534,Jeux (Salle de),unsigned,unclassified,Histoire,"Jeux (Salle de). Voyez Théatre, Amphithéatre, &c."
65379,15,3886,TAGETES,Jaucourt,Botanique,Histoire naturelle,"TAGETES, s. m. (Botan.) Tournefort distingue d..."
50736,12,3090,Plainte,Boucher d'Argis,Jurisprudence,Droit Jurisprudence,"Plainte, s. f. (Jurisprud.) est une déclaratio..."
50407,12,2761,Pincer un livre,unsigned,Relieur,Beaux-arts,"Pincer un livre, (terme de Relieur.) c'est app..."
33244,8,2396,Identité,unsigned,Grammaire,Philosophie,"Identité, (Gramm.) terme introduit récemment d..."


## Traitement des textes par le LLM

In [19]:
import ollama
import difflib
from tqdm import tqdm

In [20]:
model_names = ['gemma3:4b','deepseek-r1:8b','llama3.1:8b','mistral:7b']
model_name = model_names[0]

In [21]:

preprompt = "Corrige l’orthographe de ce texte en respectant strictement les contraintes suivantes : " \
"1. utiliser une orthographe moderne et standardisée, " \
"2. conserver le vocabulaire, " \
"3. conserver la ponctuation, " \
"4. ne pas supprimer ou ajouter de virgules, " \
"5. ne pas modifier les majuscules, " \
"6. corriger les noms propres lorsque cela est nécessaire. "


In [22]:
model_names = ['gemma3:4b','deepseek-r1:8b','llama3.1:8b','mistral:7b']
model_name = model_names[0]
tqdm.pandas(desc=f"Processing text for {model_name}")
df_sample["text_" + model_name] = df_sample["text"].progress_apply(lambda x: ollama.generate(model=model_name, prompt=preprompt+f"\nVoici le texte :\n\"\"\"\n{x}\n\"\"\"")['response'])


Processing text for gemma3:4b: 100%|██████████| 5/5 [01:29<00:00, 17.90s/it]


In [23]:
df_sample.head()

,volume,numero,head,author,artfl_domain,geode_domain,text,text_gemma3:4b
33382,8,2534,Jeux (Salle de),unsigned,unclassified,Histoire,"Jeux (Salle de). Voyez Théatre, Amphithéatre, &c.","Jeux (Salle de). Voyez Théâtre, Amphithéâtre, &c."
65379,15,3886,TAGETES,Jaucourt,Botanique,Histoire naturelle,"TAGETES, s. m. (Botan.) Tournefort distingue d...","TAGETES, s. m. (Botan.) Tournefort distingue d..."
50736,12,3090,Plainte,Boucher d'Argis,Jurisprudence,Droit Jurisprudence,"Plainte, s. f. (Jurisprud.) est une déclaratio...","Plainte, s. f. (Jurisprud.) est une déclaratio..."
50407,12,2761,Pincer un livre,unsigned,Relieur,Beaux-arts,"Pincer un livre, (terme de Relieur.) c'est app...",Pincer un livre (terme de relieur) c’est appro...
33244,8,2396,Identité,unsigned,Grammaire,Philosophie,"Identité, (Gramm.) terme introduit récemment d...","Identité (Gramm.), terme introduit récemment d..."


## Export du dataframe vers un fichier CSV


In [25]:
df_sample.to_excel("output/sample_avec_normalisation.xlsx", index=False)

## Création des fichiers HTML pour la visualisation des différences

In [29]:
# Création d'un comparateur
d = difflib.HtmlDiff(wrapcolumn=60)

def create_diff(id, input_txt, result, model_name):
    # Génération du diff en HTML
    html_diff = d.make_file(
        input_txt.splitlines(),
        result.splitlines(),
        fromdesc="Texte original",
        todesc="Texte modifié"
    )

    with open(f"./diff/diff_{model_name}_{id}.html", "w", encoding="utf-8") as f:
        f.write(html_diff)

tqdm.pandas(desc=f"Creating diffs for {model_name}")
df_sample.progress_apply(lambda row: create_diff(str(row['volume'])+"_"+str(row['numero'])+"_"+row['head'], row['text'], row["text_" + model_name], model_name), axis=1)

Creating diffs for gemma3:4b: 100%|██████████| 5/5 [00:00<00:00, 48.07it/s]


33382    None
65379    None
50736    None
50407    None
33244    None
dtype: object